<a href="https://colab.research.google.com/github/P-Ragul/Sales_Prediction/blob/main/Sales_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from os.path import exists
import pandas as pd

df_path = "train.csv.zip" if exists("train.csv.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df = pd.read_csv(df_path)
df = df.sample(n=19_000, random_state=0)
df["store"] = df["store"].astype(str)
df["item"] = df["item"].astype(str)
df["date"] = pd.to_datetime(df["date"])
df.sort_values("date", inplace = True)
df.reset_index(inplace=True,drop = True)
df.head()

,date,store,item,sales
0,2013-01-01,7,5,5
1,2013-01-01,4,9,19
2,2013-01-01,1,33,37
3,2013-01-01,3,41,14
4,2013-01-01,5,24,26


In [ ]:
train = df[df["date"] < "2017-01-01"]
test = df[df["date"] >= "2017-01-01"]

In [ ]:
train_features = train.drop(columns=["sales"])
train_target = train["sales"]
test_features = test.drop(columns=["sales"])
test_target = test["sales"]

Enrich Features

In [ ]:
pip install upgini

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 78 kB 3.2 MB/s 
     |████████████████████████████████| 1.5 MB 30.9 MB/s 
     |████████████████████████████████| 76.6 MB 102 kB/s 
     |████████████████████████████████| 2.0 MB 35.4 MB/s 


In [ ]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher(
    search_keys = {
        "date": SearchKey.DATE,
    },
    cv =CVType.time_series
)
enricher.fit(train_features,
             train_target,
             eval_set=[(test_features,test_target)]) 


Detected task type: ModelTaskType.REGRESSION


Column name,Status,Description
date,All valid,All values in this column are good to go
target,All valid,All values in this column are good to go


Running search request with search_id=99cd22c0-220a-4508-ad12-0d3fa42112ac
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
Done


28 relevant feature(s) found with the search keys: ['date'].


,feature_name,shap_value,coverage %,type
0,item,0.487726,100.0,CHARACTER
1,store,0.172106,100.0,CHARACTER
2,f_weather_date_weather_pca_0_d7e0a1fc,0.056047,100.0,NUMERIC
3,f_events_date_week_sin1_847b5db1,0.044632,100.0,NUMERIC
4,f_events_date_week_cos1_f6a8c1fc,0.029552,100.0,NUMERIC
5,f_weather_date_weather_umap_48_b39cd0c4,0.025132,100.0,NUMERIC
6,f_weather_date_weather_umap_24_2e14c9a6,0.019315,100.0,NUMERIC
7,f_weather_date_weather_umap_33_89bb7578,0.014638,100.0,NUMERIC
8,f_events_date_year_cos1_9014a856,0.012112,100.0,NUMERIC
9,f_financial_date_dow_jones_fe02128f,0.007461,100.0,NUMERIC


In [ ]:
from catboost.core import CatBoostRegressor
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)
enricher.calculate_metrics(
    train_features, train_target,
    eval_set=[(test_features, test_target)],
    estimator = model,
    scoring = "mean_absolute_percentage_error"
)

Calculating metrics...
Done


,match_rate,baseline mean_absolute_percentage_error,enriched mean_absolute_percentage_error,uplift
,,,,
train,100.0,0.255464,0.172231,0.083233
eval 1,100.0,0.268370,0.187433,0.080937


In [ ]:
enriched_train_features = enricher.transform(train_features,keep_input = True)
enriched_test_features = enricher.transform(test_features,keep_input = True)
enriched_train_features.head()

90.39637% of the rows are fully duplicated


Column name,Status,Description
date,All valid,All values in this column are good to go


Running search request with search_id=b82e27eb-7d9a-48f4-98cb-45890b0db272
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
Done

Done
90.36176% of the rows are fully duplicated


Column name,Status,Description
date,All valid,All values in this column are good to go


Running search request with search_id=c4bee194-4015-42f2-ae2b-8ffdfa980599
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
Done

Done


,date,store,item,f_weather_date_weather_pca_0_d7e0a1fc,f_events_date_week_sin1_847b5db1,f_events_date_week_cos1_f6a8c1fc,f_weather_date_weather_umap_48_b39cd0c4,f_weather_date_weather_umap_24_2e14c9a6,f_weather_date_weather_umap_33_89bb7578,f_events_date_year_cos1_9014a856,...,f_events_date_italy_game_cnt_99570b80,f_financial_date_nasdaq_c568533e,f_weather_date_weather_umap_34_c3ef5b4f,f_economic_date_cbpol_umap_1_7eb7a343,f_economic_date_cbpol_pca_3_27450634,f_financial_date_silver_7d_to_1y_b364a149,f_economic_date_cbpol_umap_6_aa0352de,f_events_date_mlending_approve_score_427a77b3,f_weather_date_weather_umap_45_d474bf8d,f_financial_date_silver_7d_to_7d_1y_shift_9cb6bdfc
0,2013-01-01,7,5,29.676683,0.781831,0.62349,4.540985,5.828106,4.644803,0.98522,...,0,3019.51001,5.664261,4.815701,-0.323471,0.963323,1.367325,0.338412,4.923654,1.072025
1,2013-01-01,4,9,29.676683,0.781831,0.62349,4.540985,5.828106,4.644803,0.98522,...,0,3019.51001,5.664261,4.815701,-0.323471,0.963323,1.367325,0.338412,4.923654,1.072025
2,2013-01-01,1,33,29.676683,0.781831,0.62349,4.540985,5.828106,4.644803,0.98522,...,0,3019.51001,5.664261,4.815701,-0.323471,0.963323,1.367325,0.338412,4.923654,1.072025
3,2013-01-01,3,41,29.676683,0.781831,0.62349,4.540985,5.828106,4.644803,0.98522,...,0,3019.51001,5.664261,4.815701,-0.323471,0.963323,1.367325,0.338412,4.923654,1.072025
4,2013-01-01,5,24,29.676683,0.781831,0.62349,4.540985,5.828106,4.644803,0.98522,...,0,3019.51001,5.664261,4.815701,-0.323471,0.963323,1.367325,0.338412,4.923654,1.072025


In [ ]:
model.fit(train_features,train_target)
preds = model.predict(test_features)
eval_metric(test_target.values, preds, "SMAPE" )

[37.65141857448004]

In [ ]:
model.fit(enriched_train_features,train_target)
enriched_preds = model.predict(enriched_test_features)
eval_metric(test_target.values, enriched_preds, "SMAPE" )

[14.50094486771499]